In [1]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 14.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 12.9 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical

2023-04-21 07:40:28.161370: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
!nvidia-smi

Fri Apr 21 07:40:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.05    Driver Version: 528.24       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 38%   36C    P8    17W / 215W |    844MiB /  8192MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# Import data:
# 1. read the text file line by line;
# 2. format the data in DataFrame.

def read_data(path):
    data_list = []
    with open(path, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            d_str = line.split()
            d_tem = [float(d) for d in d_str]
            data_list.append(d_tem)
    data = pd.DataFrame(data_list)
    return data.T

# ready data for training:
# 1. sample_size=100: the most 100 recent updates
# 2. feature_num=40: 40 features per time stamp
# 3. target_num=5: relative changes for the next 1,2,3,5 and 10 events(5 in total)
def get_model_data(data, sample_size=100, feature_num=40, target_num=5):
    data = data.values
    shape = data.shape
    X = np.zeros((shape[0]-sample_size, sample_size, feature_num))
    Y = np.zeros(shape=(shape[0]-sample_size, target_num))
    for i in range(shape[0]-sample_size):
        X[i] = data[i:i+sample_size,0:feature_num]# take the first 40 columns as features
        Y[i] = data[i+sample_size-1,-target_num:]# take the last 5 columns as labels
    X = X.reshape(X.shape[0], sample_size, feature_num, 1)# add the 4th dimension: 1 channel
    
    # "Benchmark dataset for mid-price forecasting of limit order book data with machine learning"
    # labels 1: equal to or greater than 0.002
    # labels 2: -0.00199 to 0.00199
    # labels 3: smaller or equal to -0.002
    # Y=Y-1 relabels as 0,1,2
    Y = Y-1
    return X,Y
    
        

In [5]:
#data_path = r'C:\Users\admin\OneDrive\ドキュメント\Data\BenchmarkDatasets\BenchmarkDatasets\NoAuction\1.NoAuction_Zscore\NoAuction_Zscore_Training\Train_Dst_NoAuction_ZScore_CF_9.txt'
data_path = r'./Train_Dst_NoAuction_ZScore_CF_9.txt'

data = read_data(data_path)
train_X, train_Y = get_model_data(data)
train_Y = train_Y.astype(int)


In [6]:
train_X.shape

(362300, 100, 40, 1)

In [7]:
# the size of a single input is (100,40)
input_tensor = Input(shape=(100,40,1))

# convolutional filter is (1,2) with stride of (1,2)
layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(input_tensor)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

layer_x = layers.Conv2D(16, (1,10))(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

# Inception Module
tower_1 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)
tower_1 = layers.Conv2D(32, (3,1), padding='same')(tower_1)
tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)

tower_2 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)
tower_2 = layers.Conv2D(32, (5,1), padding='same')(tower_2)
tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)  

tower_3 = layers.MaxPooling2D((3,1), padding='same', strides=(1,1))(layer_x)
tower_3 = layers.Conv2D(32, (1,1), padding='same')(tower_3)
tower_3 = layers.LeakyReLU(alpha=0.01)(tower_3)

layer_x = layers.concatenate([tower_1, tower_2, tower_3], axis=-1)

# concatenate features of tower_1, tower_2, tower_3
layer_x = layers.Reshape((100,96))(layer_x)

# 64 LSTM units
layer_x = LSTM(64)(layer_x)
# The last output layer uses a softmax activation function
output = layers.Dense(3, activation='softmax')(layer_x)
model = Model(input_tensor, output)

model.summary()

2023-04-21 07:44:40.900910: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-21 07:44:40.905297: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-21 07:44:40.905583: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-21 07:44:40.906606: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-21 07:44:40.906835: I tensorflow/compile

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 40, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 100, 20, 16)  48          ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 100, 20, 16)  0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 100, 20, 16)  1040        ['leaky_re_lu[0][0]']        

2023-04-21 07:45:19.149077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-21 07:45:19.150785: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-21 07:45:19.152371: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01, epsilon=1)# learning rate and epsilon are the same as paper DeepLOB
y = to_categorical(train_Y[:,0])# y is the next event's mid price (k=1)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(train_X, y, epochs=100, batch_size=32)

2023-04-21 07:45:35.000605: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5796800000 exceeds 10% of free system memory.
2023-04-21 07:45:56.315904: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5796800000 exceeds 10% of free system memory.


Epoch 1/100


2023-04-21 07:45:58.709353: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-21 07:45:58.711067: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-21 07:45:58.712782: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

11322/11322 [==============================] - 214s 18ms/step - loss: 0.8770 - accuracy: 0.6424
Epoch 2/100
11322/11322 [==============================] - 196s 17ms/step - loss: 0.7353 - accuracy: 0.6836
Epoch 3/100
11322/11322 [==============================] - 200s 18ms/step - loss: 0.6726 - accuracy: 0.7086
Epoch 4/100
11322/11322 [==============================] - 195s 17ms/step - loss: 0.6469 - accuracy: 0.7355
Epoch 5/100
11322/11322 [==============================] - 195s 17ms/step - loss: 0.5973 - accuracy: 0.7691
Epoch 6/100
11322/11322 [==============================] - 197s 17ms/step - loss: 0.5823 - accuracy: 0.7761
Epoch 7/100
11322/11322 [==============================] - 196s 17ms/step - loss: 0.5727 - accuracy: 0.7801
Epoch 8/100
11322/11322 [==============================] - 196s 17ms/step - loss: 0.5654 - accuracy: 0.7822
Epoch 9/100
11322/11322 [==============================] - 197s 17ms/step - loss: 0.5583 - accuracy: 0.7858
Epoch 10/100
  628/11322 [>.............

In [ ]:
#test_data.to_csv('FI2010_test.csv')
data.to_csv('FI2010_test.csv')

In [ ]:
#test_path = r'E:\JupyterFile\BenchmarkDatasets\BenchmarkDatasets\NoAuction\1.NoAuction_Zscore\NoAuction_Zscore_Testing\Test_Dst_NoAuction_ZScore_CF_9.txt'
test_path = r'./Test_Dst_NoAuction_ZScore_CF_9.txt'
test_data = read_data(test_path)

In [ ]:
#test_data = read_data(test_path)
test_X, test_Y = get_model_data(test_data)
test_Y = test_Y.astype(int)
test_y = to_categorical(test_Y[:,0])

model.evaluate(test_X, test_Y)

In [ ]:
test_Y